<a href="https://colab.research.google.com/github/juliana-zh/ML/blob/main/NLP_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Обучение модели с использованием представления слов (embeddings)

Загружаем датасет из новостных текстов, состоящих из 20 групп

In [30]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)
newsgroups_train.filenames.shape

(2034,)

In [31]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Загружаем предварительно натренированные представления слов:

In [32]:
import gensim.downloader as api
embeddings_pretrained = api.load('glove-twitter-25')

Для каждого текста из нашей тренировочной выборки получим набор слов.
По набору слов с помощью технологии Word2Vec получим представления этих слов.


Функция, отвечающая за предобработку:

In [33]:
# стоп-слова, preproc

stopWords = set(stopwords.words('english'))
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

def preproc_nltk(text):
    #text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])

st = "Oh, I think I ve landed Where there are miracles at work,  For the thirst and for the hunger Come the conference of birds"
preproc_nltk(st)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'oh , think landed miracle work , thirst hunger come conference bird'

Здесь мы работаем над вариантом, когда мы используем технологию Word2Vec, чтобы подсчитать представления слов.

In [34]:
from gensim.models import Word2Vec

proc_words = [preproc_nltk(text).split() for text in newsgroups_train.data]
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                 vector_size=100,         # embedding vector size
                 min_count=3,             # consider words that occured at least 5 times
                 window=3).wv

Здесь мы работаем над вариантом, когда нам *даны заранее подсчитанные представления слов*. Создаем функцию, которая является отображением из какого-либо комментария в сумму векторов, являющихся представлениями слов комментария:

In [35]:
def vectorize_sum(comment, embeddings):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')

    for word in preproc_nltk(comment).split():
        if word in embeddings:
            features += embeddings[f'{word}']

    return features

In [36]:
len(embeddings_trained.index_to_key)

13566

Разбиваем выборку на обучающую и тестовую

In [37]:
X_wv = np.stack([vectorize_sum(text, embeddings_pretrained) for text in newsgroups_train.data])
X_train_wv, X_test_wv, y_train, y_test = train_test_split(X_wv, newsgroups_train.target, test_size=0.2, random_state=0)
X_train_wv.shape, X_test_wv.shape

((1627, 25), (407, 25))

Обучаем модель логистической регрессии, считаем точность:

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

clf = LogisticRegression(max_iter=5000)
wv_model = clf.fit(X_train_wv, y_train)
accuracy_score(y_test, wv_model.predict(X_test_wv))

0.7027027027027027

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

clf = LogisticRegression(max_iter=10000)
wv_model = clf.fit(X_train_wv, y_train)
accuracy_score(y_test, wv_model.predict(X_test_wv))

0.7027027027027027